In [170]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('./scripts')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
from certificate import Certificate
from blockchain import Blockchain
from smart_contract import SmartContractDefinition, SmartContractWritingOperation
from network import Node
from node import BlockchainNode
from wallet import Wallet
from proof_of_stake import ProofOfStake
from block import Block
from pebble import Account
from time import sleep
from helpers import timestamp

In [176]:
walletNode1=Wallet()
walletNode2=Wallet()
walletNode3=Wallet()

Bob=Wallet()
BobAccount = Account(Bob.publicKey)
print('Bob has ', BobAccount.pebbleAmount, ' pebbles')
John=Wallet()
JohnAccount = Account(John.publicKey)
print('John has ', JohnAccount.pebbleAmount, ' pebbles')
Alice=Wallet()
AliceAccount = Account(Alice.publicKey)
print('Alice has ', AliceAccount.pebbleAmount, ' pebbles')
Emma=Wallet()
EmmaAccount = Account(Alice.publicKey)
print('Emma has ', EmmaAccount.pebbleAmount, ' pebbles')

pos = ProofOfStake(walletNode1.publicKey)

def reset_blockchain():
    Node.reset_network()
    global node1, node2, node3
    node1 = BlockchainNode(walletNode1, pos)
    node2 = BlockchainNode(walletNode2, pos)
    node3 = BlockchainNode(walletNode3, pos)

Bob has  20  pebbles
John has  20  pebbles
Alice has  20  pebbles
Emma has  20  pebbles


In [177]:
def read_text_file(fileName):
    with open(fileName, 'r', encoding='utf-8') as f:
        return f.read()
    
betFilePath="./scripts/bet.py"    


For testing we will work on the contract through the instantiate function for ease of use. However, for the showcase in class, we must use the ContractWritingOperation and set it all up in the blockchain.

In this example below, we showcase multiple bets going on at once.

In [184]:
reset_blockchain()

bet1=SmartContractDefinition(Alice.publicKey,read_text_file(betFilePath))
Alice.sign(bet1)
node1.new_certificate(bet1)
sleep(1)



bet1_start = SmartContractWritingOperation(Alice.publicKey, bet1.hash(), 'startBet', [timestamp.now()+timestamp.now(), 'Lorum Lora', ['yes', 'no', 'maybe']])
Alice.sign(bet1_start)
node1.new_certificate(bet1_start)
sleep(1)

cert1 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert1)
node1.new_certificate(cert1)

cert2 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert2)
node1.new_certificate(cert2)

cert3 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert3)
node1.new_certificate(cert3)

cert4 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert4)
node1.new_certificate(cert4)

cert5 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert5)
node1.new_certificate(cert3)

cert6 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert6)
node1.new_certificate(cert6)

#node1.print_certbox()
#node1.blockchain.display()

SmartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(node1.blockchain, bet1.hash())
SmartContractObject.logDump()

print('John has ', JohnAccount.pebbleAmount, ' pebbles')


#contract1.startBet(Alice.publicKey,1,5,"Lorum Lora",["yes","no","maybe"])
#contract2.startBet(Emma.publicKey,1,5,"Arnoe Uytlel",["yes","no","maybe"])

#contract1.display()
#contract2.display()


[node 3082.c23a.0001] Adding
{'Public Key': '30820122', 'Signature': '93b38689', 'timestamp': 1708529258025, 'type': <class 'smart_contract.SmartContractDefinition'>}
[node 3082.c23a.0001] I am sending the data "<smart_contract.SmartContractDefinition object at 0x00000226A7F54FD0>" to EVERYBODY!
[node 3082.c23a.0001] I am sending the data "<smart_contract.SmartContractDefinition object at 0x00000226A7F54FD0>" to "node 3082.0560.0001"
[node 3082.0560.0001] Adding
{'Public Key': '30820122', 'Signature': '93b38689', 'timestamp': 1708529258025, 'type': <class 'smart_contract.SmartContractDefinition'>}
[node 3082.0560.0001] I am sending the data "<smart_contract.SmartContractDefinition object at 0x00000226A7F54FD0>" to EVERYBODY!
[node 3082.0560.0001] I am sending the data "<smart_contract.SmartContractDefinition object at 0x00000226A7F54FD0>" to "node 3082.c23a.0001"
[node 3082.c23a.0001] 3 :
{'Public Key': '30820122', 'Signature': '93b38689', 'timestamp': 1708529258025, 'type': <class 'sm

In [183]:
print(read_text_file(betFilePath))

class SmartContract():
    def __init__(self,ownerPublicKey):
        self.ownerPublicKey=ownerPublicKey
        self.description=None
        self.startTime=None
        self.closingPeriod=None
        #dictionnary of betters with their public key, the amount staked, and the outcome they want
        self.betters=[]
        #list of strings for different outcomes
        self.outcomes=[]
        self.logs=[]
        self.type="BET"

#these two functions are to be called outside blockchain only_______________    
    def display(self,*args):
        print(self.description)
        for i,o in enumerate(self.outcomes):
            print(f'Select {i+1} for {o}')
        print(self.closingPeriod)
        print('Current betters---------')
        total={}
        for b in self.betters:
            print(f"User {b['PublicKey'][256:264]} bet {b['Amount']} on {b['Outcome']}.")
            if b['Outcome'] in total.keys():
                total[b['Outcome']]+=b['Amount']
            else:
      

A solution to the timestamp issue in SmartContract will be to use the SmartCOntractWritingOperation wrapper to pass a timestamp everytime it calls a function.

In [115]:
#contract1.addBetter(JohnAccount,3,20,1)


#contract1.logDump()

John has  20  pebbles


In [106]:
print('John has ', JohnAccount.pebbleAmount, ' pebbles')

John has  20  pebbles


In [107]:
#contract1.closeBet(Alice.publicKey,1, 1)
cert2 = SmartContractWritingOperation(Alice.publicKey, bet1.hash(), 'closeBet', 1)
John.sign(cert1)

In [108]:
print('John has ', JohnAccount.pebbleAmount, ' pebbles')

John has  20  pebbles
